NAS implementation


Genetic Algorithm for find the best neural architecture

Create population

In [1]:
import random
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
type_feature_extraction = "feature extraction"
hidden_layer = "hidden layer"
type_activation_function = "activation function"
fitness = 'fitness'

num_of_generations = 10
num_of_chromosomes = 10

num_epochs = 5
num_exe = 5

crossover_rate = 0.8 # select parents for crossover with this rate
mutation_rate = 0.01

In [3]:
def create_chromosomes(num_chromosomes):
    chromosomes = []
    for i in range(num_chromosomes):
        chro = dict()
        chro[type_feature_extraction] = random.randint(1,3)
        num_hidden_layer = random.randint(0, 2)
        chro[hidden_layer] = [random.choice([10, 20, 30]) for _ in range(num_hidden_layer)]
        chro[type_activation_function] = [random.choice([1, 2]) for _ in range(num_hidden_layer + 1)]
        chro[fitness] = 0.0
        chromosomes.append(chro)

    return chromosomes

In [4]:
print(create_chromosomes(4))

[{'feature extraction': 1, 'hidden layer': [30, 30], 'activation function': [2, 2, 2], 'fitness': 0.0}, {'feature extraction': 1, 'hidden layer': [], 'activation function': [2], 'fitness': 0.0}, {'feature extraction': 2, 'hidden layer': [30, 30], 'activation function': [2, 1, 1], 'fitness': 0.0}, {'feature extraction': 2, 'hidden layer': [30], 'activation function': [2, 2], 'fitness': 0.0}]


## Models

In [5]:
class MLP_0hidden(nn.Module):
    def __init__(self, input_size, output_size, activation):
        super(MLP_0hidden, self).__init__()
        self.fc1 = nn.LazyLinear(output_size)
        # self.activation_output = nn.ReLU() if torch.rand(1) > 0.5 else nn.Sigmoid()
        if activation[0] == 1:
            self.activation_input = nn.ReLU()
        elif activation[0] == 2:
            self.activation_input = nn.Sigmoid()
        self.activation_output = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_output(x)
        return x
    

class MLP_1hidden(nn.Module):
    def __init__(self, hidden_layer, activations):
        hidden_size = hidden_layer[0]
        output_size = 10
        super(MLP_1hidden, self).__init__()
        self.fc1 = nn.LazyLinear(hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        if activations[0] == 1:
            self.activation_input = nn.ReLU()
        elif activations[0] == 2:
            self.activation_input = nn.Sigmoid()
        
        if activations[1] == 1:
            self.activation_hidden1 = nn.ReLU()
        elif activations[1] == 2:
            self.activation_hidden1 = nn.Sigmoid()
        
        self.activation_output  = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_hidden1(x)
        x = self.fc2(x)
        x = self.activation_output(x)
        return x
    


class MLP_2hidden(nn.Module):
    def __init__(self, hidden_layer, activations):
        super(MLP_2hidden, self).__init__()
        hidden1_size = hidden_layer[0]
        hidden2_size = hidden_layer[1]
        output_size = 10
        self.fc1 = nn.LazyLinear(hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, output_size)
        
        if activations[0] == 1:
            self.activation_input = nn.ReLU()
        elif activations[0] == 2:
            self.activation_input = nn.Sigmoid()
            
        if activations[1] == 1 :
            self.activation_hidden1 = nn.ReLU()
        elif activations[1] == 2:
            self.activation_hidden1 = nn.Sigmoid()
        
        if activations[2] == 1:
            self.activation_hidden2 = nn.ReLU()
        elif activations[2] == 2:
            self.activation_hidden2 = nn.Sigmoid()
        
        
        
        self.activation_output = nn.Softmax(dim=1)        

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_hidden1(x)
        x = self.fc2(x)
        x = self.activation_hidden2(x)
        x = self.fc3(x)
        x = self.activation_output(x)
        return x

In [ ]:
torch.manual_seed(42)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_list = list()
for images, labels in train_loader:
        img = images.to(device)  # Move input tensor to the same device as the model
        lbl = labels.to(device)  # Move label tensor to the same device as the model

        # Feature extraction using the pre-trained model
        with torch.no_grad():
            feature_extraction = models.resnet18(pretrained=True)
            feature_extraction = nn.Sequential(*list(feature_extraction.children())[:-1])  # Remove the last fully connected layer
            feature_extraction = feature_extraction.to(device)            # Move feature extraction model to the same device as the model
            features1 = feature_extraction(img)
            features1 = features1.view(features1.size(0), -1)

            feature_extraction = models.resnet34(pretrained=True)
            feature_extraction = nn.Sequential(*list(feature_extraction.children())[:-1])  # Remove the last fully connected layer
            feature_extraction = feature_extraction.to(device)            # Move feature extraction model to the same device as the model
            features2 = feature_extraction(img)
            features2 = features2.view(features2.size(0), -1)
            
            feature_extraction = models.vgg11(pretrained=True)
            feature_extraction = nn.Sequential(*list(feature_extraction.children())[:-1])  # Remove the last fully connected layer
            feature_extraction = feature_extraction.to(device)            # Move feature extraction model to the same device as the model
            features3 = feature_extraction(img)
            features3 = features3.view(features3.size(0), -1)

        feature_list.append([features1, features1, features1])


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)
        
criterion = nn.CrossEntropyLoss()


def train(model, chromosome):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    if torch.cuda.is_available():
        model = model.cuda() 

    train_accuracy = list()

    for exe in range(num_exe):
        for epoch in range(num_epochs):
            model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        i=-1
        if chromosome[type_feature_extraction] == 1:
            i = 0
        elif chromosome[type_feature_extraction] == 2:
            i = 1
        elif chromosome[type_feature_extraction] == 3:
            i = 2

        for feature in feature_list:
            # Forward pass
            outputs = model(feature[i])
            loss = criterion(outputs, lbl)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += lbl.size(0)
            correct += predicted.eq(lbl).sum().item()

        train_loss /= len(train_loader)
        train_accuracy.append(correct / total)

    # print(f"Epoch [{epoch + 1}/{num_epochs}], "
    #       f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

    return sum(train_accuracy)/len(train_accuracy)
    


def feed_Data(chromosomes):
    input_size = 512  # Size of the ResNet output feature vector
    output_size = 10  # Number of classes in CIFAR-10 dataset
    for i in range(len(chromosomes)):
        if len(chromosomes[i][hidden_layer]) == 0:
            model = MLP_0hidden(input_size, output_size, chromosomes[i][type_activation_function])
        elif len(chromosomes[i][hidden_layer]) == 1:
            model = MLP_1hidden(input_size, chromosomes[i][hidden_layer], chromosomes[i][type_activation_function])
        elif len(chromosomes[i][hidden_layer]) == 2:
            model = MLP_2hidden(input_size, chromosomes[i][hidden_layer], chromosomes[i][type_activation_function])
        
        chromosomes[i][fitness] = train(model, chromosomes[i])
    
    return chromosomes

100%|██████████| 170498071/170498071 [00:02<00:00, 77513267.56it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

In [ ]:
def evaluate_fitness_population(population):
    feed_Data(population)
    return population

In [ ]:
def select_parents(population):
    num_selected = int(crossover_rate*len(population))
    return population[:num_selected]

In [ ]:
def crossover_feature(parent1, parent2):
    parent1 , parent2 = parent2 , parent1
    return parent1 , parent2

In [ ]:
def crossover_layer_activation(parent1_layer, parent2_layer, parent1_active, parent2_active):
    parent1_len = len(parent1_layer)
    parent2_len = len(parent2_layer)
    
    if parent1_len - parent2_len == -1:
        value = parent2_layer.pop()
        parent1_layer.append(value)
        value = parent2_active.pop()
        parent1_active.append(value)

    elif parent1_len - parent2_len == 1:
        value = parent1_layer.pop()
        parent2_layer.append(value)
        value = parent1_active.pop()
        parent2_active.append(value)

    elif parent1_len == parent2_len and parent1_len != 0:
        parent1_layer[-1], parent2_layer[-1] = parent2_layer[-1], parent1_layer[-1]
        parent1_active[-1], parent2_active[-1] = parent2_active[-1], parent1_active[-1]

    elif parent1_len == 0 and parent2_len == 0:
        pass
    
    else:
        if parent2_len > parent1_len:
            value = parent2_layer.pop()
            parent1_layer.append(value)
            value = parent2_active.pop()
            parent1_active.append(value)
        else:
            value = parent1_layer.pop()
            parent2_layer.append(value)
            value = parent1_active.pop()
            parent2_active.append(value)
    
    
    return parent1_layer, parent2_layer, parent1_active, parent2_active

In [ ]:
def crossover(parents):
    offspring = list()
    for i in range(int(len(parents)/2)):
        child1 = dict()
        child2 = dict()

        child1[type_feature_extraction], child2[type_feature_extraction] = crossover_feature(parents[2*i][type_feature_extraction], parents[2*i+1][type_feature_extraction])
        child1[hidden_layer], child2[hidden_layer], child1[type_activation_function], child2[type_activation_function] = crossover_layer_activation(parents[2*i][hidden_layer], parents[2*i+1][hidden_layer],parents[2*i][type_activation_function], parents[2*i+1][type_activation_function])

        child1[fitness] = 0
        child2[fitness] = 0
        
        offspring.append(child1)
        offspring.append(child2)
    
    return offspring

In [ ]:
def mutation_feature(parent):
    parent = random.randint(1,3)
    return parent

In [ ]:
def mutation_activition(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.randint(1,2)
    return parent

In [ ]:
def mutation_layer(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.choice([10, 20, 30])
    return parent

In [ ]:
def mutate(offspring):
    for child in offspring:
        if random.uniform(0, 1) <= mutation_rate:
            child[type_feature_extraction] = mutation_feature(child[type_feature_extraction])
        if random.uniform(0, 1) <= mutation_rate:
            child[type_activation_function] = mutation_activition(child[type_activation_function])
        if random.uniform(0, 1) <= mutation_rate:
            child[hidden_layer] = mutation_layer(child[hidden_layer])
    
    return offspring

In [ ]:
def select_survivors(population, offspring):
    while(len(offspring) > 0):
        best_child = max(offspring, key=lambda x: x[fitness])
        bad_parent = min(population, key=lambda x: x[fitness])
        if best_child[fitness] > bad_parent[fitness]:
            population.remove(bad_parent)
            population.append(best_child)
            offspring.remove(best_child)
        else:
            break
    
    return population

In [ ]:
def genetic_algorithm():
    # Initialize population
    population = create_chromosomes(num_of_chromosomes)

    # Evaluate population fitness
    # fitness = accuracy of the model 
    evaluate_fitness_population(population)

    # Sort population
    population.sort(key = lambda x: x[fitness])

    # While termination condition is not met
    for _ in range(num_of_generations):
        # Select parents
        parents = select_parents(population)

        # Apply crossover / mutation operators
        offspring = crossover(parents)
        mutate(offspring)

        # Evaluate offspring
        evaluate_fitness_population(offspring)

        # Select survivors
        population = select_survivors(population, offspring)

        # Sort population
        population.sort(key = lambda x: x[fitness])

    return max(population, key=lambda x: x[fitness])   # Return best individual in population

In [ ]:
#print(genetic_algorithm())
m = genetic_algorithm()
print(m)